In [1]:
# (1) パッケージをインストールする（初回のみ）
install.packages("broom")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
# (2) ライブラリの読み出し
library("tidyverse")
library("broom")

── Attaching packages ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.1.2     ✔ dplyr   1.0.7
✔ tidyr   1.1.3     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



In [3]:
# (3) データの読み込み
email_data <- read_csv("http://www.minethatdata.com/Kevin_Hillstrom_MineThatData_E-MailAnalytics_DataMiningChallenge_2008.03.20.csv")

# (4) 女性向けメールが配信されたデータを削除したデータを作成
male_df <- email_data %>%
  filter(segment != "Womens E-Mail") %>% # 女性向けメールが配信されたデータを削除
  mutate(treatment = ifelse(segment == "Mens E-Mail", 1, 0)) # 介入を表すtreatment変数を追加

# (5) セレクションバイアスのあるデータを作成
## seedを固定
set.seed(1)

## 条件に反応するサンプルの量を半分にする
obs_rate_c <- 0.5
obs_rate_t <- 0.5

## バイアスのあるデータを作成
biased_data <- male_df %>%
  mutate(obs_rate_c =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), obs_rate_c, 1),
         obs_rate_t =
           ifelse( (history > 300) | (recency < 6) |
                     (channel == "Multichannel"), 1, obs_rate_t),
         random_number = runif(n = NROW(male_df))) %>%
  filter( (treatment == 0 & random_number < obs_rate_c ) |
            (treatment == 1 & random_number < obs_rate_t) )

head(email_data)


── Column specification ──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
cols(
  recency = col_double(),
  history_segment = col_character(),
  history = col_double(),
  mens = col_double(),
  womens = col_double(),
  zip_code = col_character(),
  newbie = col_double(),
  channel = col_character(),
  segment = col_character(),
  visit = col_double(),
  conversion = col_double(),
  spend = col_double()
)




recency,history_segment,history,mens,womens,zip_code,newbie,channel,segment,visit,conversion,spend
<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
10,2) $100 - $200,142.44,1,0,Surburban,0,Phone,Womens E-Mail,0,0,0
6,3) $200 - $350,329.08,1,1,Rural,1,Web,No E-Mail,0,0,0
7,2) $100 - $200,180.65,0,1,Surburban,1,Web,Womens E-Mail,0,0,0
9,5) $500 - $750,675.83,1,0,Rural,1,Web,Mens E-Mail,0,0,0
2,1) $0 - $100,45.34,1,0,Urban,0,Web,Womens E-Mail,0,0,0
6,2) $100 - $200,134.83,0,1,Surburban,0,Phone,Womens E-Mail,1,0,0


バイアスのあるマーケティングデータで分析してみる

モデルは、
$$ spend_i = \beta_0 + \beta_{treatment} treatment_i + \beta_{history} history_i $$

In [4]:
# (6) バイアスのあるデータでの回帰分析
## 回帰分析の実行
biased_reg <- lm(data = biased_data, formula = spend ~ treatment + history)

## 分析結果のレポート
summary(biased_reg)


Call:
lm(formula = spend ~ treatment + history, data = biased_data)

Residuals:
   Min     1Q Median     3Q    Max 
 -4.74  -1.46  -1.26  -0.48 497.74 

Coefficients:
             Estimate Std. Error t value Pr(>|t|)    
(Intercept) 0.3241996  0.1444390   2.245  0.02480 *  
treatment   0.9026109  0.1743057   5.178 2.25e-07 ***
history     0.0010927  0.0003366   3.246  0.00117 ** 
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 15.36 on 31860 degrees of freedom
Multiple R-squared:  0.001339,	Adjusted R-squared:  0.001276 
F-statistic: 21.35 on 2 and 31860 DF,  p-value: 5.406e-10


In [5]:
## 推定されたパラメーターの取り出し
biased_reg_coef <- tidy(biased_reg)
biased_reg_coef

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.324199564,0.144438990,2.244543,2.480430e-02
treatment,0.902610917,0.174305713,5.178321,2.252514e-07
history,0.001092682,0.000336606,3.246176,1.170872e-03


施策による効果量は0.9026109とある。しかし、ch1で見たように真の効果量は0.77程度なので、バイアスが生じていることがわかる。

実際にRCTの場合の回帰分析結果と比較してみよう。

In [6]:
# (7) RCTデータでの回帰分析とバイアスのあるデータでの回帰分析の比較
## RCTデータでの単回帰
rct_reg <- lm(data = male_df, formula = spend ~ treatment)
rct_reg_coef <- tidy(rct_reg)

## バイアスのあるデータでの単回帰
nonrct_reg <- lm(data = biased_data, formula = spend ~ treatment)
nonrct_reg_coef <- tidy(nonrct_reg)

In [7]:
rct_reg_coef
nonrct_reg_coef #treatmentだけ使って回帰分析(施策ごとに売上の平均をとっているのと同じ)しているともっとバイアスが大きいことがわかる

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.6527894,0.1027070,6.355841,2.093808e-10
treatment,0.7698272,0.1452479,5.300090,1.163201e-07


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.5483062,0.126891,4.321081,1.557365e-05
treatment,0.9794465,0.172717,5.670817,1.433467e-08


では次に、共変量Xを追加して分析するとどうなるか？
$$ spend_i = \beta_0 + \beta_{treatment} treatment_i + \beta_{recency}recency_i + \beta_{channel}channel_i +\beta_{history} history_i $$

In [8]:
## バイアスのあるデータでの重回帰
nonrct_mreg <- lm(data = biased_data,
                  formula = spend ~ treatment + recency + channel + history)
nonrct_mreg_coef <- tidy(nonrct_mreg)

In [9]:
biased_reg_coef
nonrct_mreg_coef

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.324199564,0.144438990,2.244543,2.480430e-02
treatment,0.902610917,0.174305713,5.178321,2.252514e-07
history,0.001092682,0.000336606,3.246176,1.170872e-03


term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.502412896,0.3793847254,1.32428341,1.854184e-01
treatment,0.846575728,0.1784759605,4.74335998,2.111119e-06
recency,-0.040266556,0.0259469894,-1.55187775,1.207014e-01
channelPhone,-0.001778911,0.3040193436,-0.00585131,9.953314e-01
channelWeb,0.226159585,0.3034664353,0.74525403,4.561237e-01
history,0.001029897,0.0003753754,2.74364500,6.079519e-03


どちらもバイアスのあるデータからの係数だが、重回帰の変数が多いほうがtreatmentの効果量が真の値に近づいている。

### 脱落変数バイアスの確認

In [10]:
# (8) OVBの確認
## (a) history抜きの回帰分析とパラメーターの取り出し
short_coef <- biased_data %>%
  lm(data = .,
     formula = spend ~ treatment + recency + channel) %>% #histroy抜きでOVBを作り出す
   tidy()
short_coef

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.09687653,0.31145956,3.5217302,4.293395e-04
treatment,0.87462971,0.17820107,4.9081058,9.241414e-07
recency,-0.05516998,0.02537463,-2.1742180,2.969610e-02
channelPhone,-0.31248934,0.28216860,-1.1074561,2.681051e-01
channelWeb,-0.08467985,0.28155374,-0.3007591,7.636002e-01


In [11]:
## aの結果から介入効果に関するパラメーターのみを取り出す
alpha_1 <- short_coef %>%
  filter(term == "treatment") %>%
  pull(estimate)
alpha_1 #P54モデルAの効果量

[1] 0.8746297

In [12]:
## (b) historyを追加した回帰分析とパラメーターの取り出し
long_coef <- biased_data %>%
  lm(data = .,
     formula = spend ~ treatment + recency + channel + history) %>% #historyも含めた回帰分析
  tidy()

## bの結果から介入とhistoryに関するパラメーターを取り出す
beta_1 <- long_coef %>% filter(term == "treatment") %>% pull(estimate)
beta_2 <- long_coef %>% filter(term == "history") %>% pull(estimate)
beta_1 #P54モデルBにおける効果量
beta_2

[1] 0.8465757

[1] 0.001029897

In [13]:
## (c) 脱落した変数と介入変数での回帰分析
omitted_coef <- biased_data %>%
  lm(data = ., formula = history ~ treatment + channel + recency) %>% #γを求める
  tidy()
## cの結果から介入変数に関するパラメーターを取り出す
gamma_1 <- omitted_coef %>% filter(term == "treatment") %>% pull(estimate)
gamma_1

[1] 27.2396

In [14]:
## OVBの確認
beta_2*gamma_1
alpha_1 - beta_1

#こんなに近くなるの不思議すぎる #でも関係式はそうなってる。これによってhistoryがないことで効果量が0.028だけ多く推定されていることがわかる


[1] 0.02805398

[1] 0.02805398

以下練習のためにbroomを利用した場合

In [15]:
# (9) OVBの確認(broomを利用した場合)
## broomの読み出し
library(broom)

## モデル式のベクトルを用意
formula_vec <- c(spend ~ treatment + recency + channel, # モデルA
               spend ~ treatment + recency + channel + history, # モデルB
               history ~ treatment + channel + recency) # モデルC

In [16]:
## formulaに名前を付ける
names(formula_vec) <- paste("reg", LETTERS[1:3], sep ="_")

In [17]:
formula_vec

$reg_A
spend ~ treatment + recency + channel

$reg_B
spend ~ treatment + recency + channel + history

$reg_C
history ~ treatment + channel + recency


In [18]:
paste("reg", LETTERS[1:3], sep ="_") #suffixをつけるみたいなもんか

[1] "reg_A" "reg_B" "reg_C"

In [19]:
LETTERS

[1] "A" "B" "C" "D" "E" "F" "G" "H" "I" "J" "K" "L" "M" "N" "O" "P" "Q" "R" "S"
[20] "T" "U" "V" "W" "X" "Y" "Z"

In [20]:
## モデル式のデータフレーム化
models = formula_vec %>%
  enframe(name = "model_index", value = "formula")
models

model_index,formula
<chr>,<list>
reg_A,spend ~ treatment + recency + channel
reg_B,spend ~ treatment + recency + channel + history
reg_C,history ~ treatment + channel + recency


In [21]:
## まとめて回帰分析を実行
df_models <- models %>% 
    mutate(model = map(.x = formula, .f = lm, data = biased_data)) %>% #回帰分析を施した
    mutate(lm_result = map(.x = model, .f = tidy))

In [22]:
tmp <- models %>% mutate(model=map(.x = formula, .f = lm, data = biased_data))

In [23]:
models %>% mutate(unko=c(1,2,3))

model_index,formula,unko
<chr>,<list>,<dbl>
reg_A,spend ~ treatment + recency + channel,1
reg_B,spend ~ treatment + recency + channel + history,2
reg_C,history ~ treatment + channel + recency,3


In [24]:
df_models$lm_result

term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),1.09687653,0.31145956,3.5217302,4.293395e-04
treatment,0.87462971,0.17820107,4.9081058,9.241414e-07
recency,-0.05516998,0.02537463,-2.1742180,2.969610e-02
channelPhone,-0.31248934,0.28216860,-1.1074561,2.681051e-01
channelWeb,-0.08467985,0.28155374,-0.3007591,7.636002e-01
term,estimate,std.error,statistic,p.value
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
(Intercept),0.502412896,0.3793847254,1.32428341,1.854184e-01
treatment,0.846575728,0.1784759605,4.74335998,2.111119e-06


In [25]:
df_models %>%
  select(formula, model_index, lm_result)

formula,model_index,lm_result
<list>,<chr>,<list>
spend ~ treatment + recency + channel,reg_A,"(Intercept) , treatment , recency , channelPhone , channelWeb , 1.09687653201018 , 0.874629708514415 , -0.0551699754020017 , -0.312489342000167 , -0.0846798533712658 , 0.311459559910636 , 0.178201070638352 , 0.0253746285630198 , 0.28216859696893 , 0.281553742724906 , 3.52173018007506 , 4.90810580083114 , -2.17421804874829 , -1.10745612855911 , -0.30075911103765 , 0.000429339535974373, 9.24141369112181e-07, 0.0296960988421261 , 0.268105131511276 , 0.763600152861394"
spend ~ treatment + recency + channel + history,reg_B,"(Intercept) , treatment , recency , channelPhone , channelWeb , history , 0.502412896129599 , 0.846575727930548 , -0.0402665557122618 , -0.00177891134775736, 0.226159585212085 , 0.00102989695461476 , 0.379384725430789 , 0.17847596054522 , 0.0259469894483822 , 0.304019343563046 , 0.303466435250855 , 0.000375375442382237, 1.32428340534562 , 4.7433599760123 , -1.55187775415588 , -0.00585130974532369, 0.745254034519943 , 2.74364499733587 , 0.1854184404305 , 2.11111876155193e-06, 0.120701410394414 , 0.995331393572634 , 0.456123661045614 , 0.00607951929013898"
history ~ treatment + channel + recency,reg_C,"(Intercept) , treatment , channelPhone , channelWeb , recency , 577.206907173491 , 27.2395995134878 , -301.690794656861 , -301.816057607065 , -14.4707872209552 , 4.64817067580048 , 2.65944314303628 , 4.21103721600024 , 4.20186123351383 , 0.378686736826441 , 124.179370215163 , 10.2425951781727 , -71.6428706710447 , -71.8291349556706 , -38.2130817208615 , 0 , 1.39489647897936e-24 , 0 , 0 , 1.96989350543756e-312"


In [26]:
## モデルの結果を整形
df_results <- df_models %>%
  mutate(formula = as.character(formula)) %>%
  select(formula, model_index, lm_result) %>%
  unnest(cols = lm_result)
df_results

formula,model_index,term,estimate,std.error,statistic,p.value
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
spend ~ treatment + recency + channel,reg_A,(Intercept),1.096877e+00,0.3114595599,3.52173018,4.293395e-04
spend ~ treatment + recency + channel,reg_A,treatment,8.746297e-01,0.1782010706,4.90810580,9.241414e-07
spend ~ treatment + recency + channel,reg_A,recency,-5.516998e-02,0.0253746286,-2.17421805,2.969610e-02
spend ~ treatment + recency + channel,reg_A,channelPhone,-3.124893e-01,0.2821685970,-1.10745613,2.681051e-01
spend ~ treatment + recency + channel,reg_A,channelWeb,-8.467985e-02,0.2815537427,-0.30075911,7.636002e-01
spend ~ treatment + recency + channel + history,reg_B,(Intercept),5.024129e-01,0.3793847254,1.32428341,1.854184e-01
spend ~ treatment + recency + channel + history,reg_B,treatment,8.465757e-01,0.1784759605,4.74335998,2.111119e-06
spend ~ treatment + recency + channel + history,reg_B,recency,-4.026656e-02,0.0259469894,-1.55187775,1.207014e-01
spend ~ treatment + recency + channel + history,reg_B,channelPhone,-1.778911e-03,0.3040193436,-0.00585131,9.953314e-01


In [27]:
## モデルA,B,Cでのtreatmentのパラメータを抜き出す
treatment_coef <- df_results %>%
  filter(term == "treatment") %>%
  pull(estimate)
treatment_coef

[1]  0.8746297  0.8465757 27.2395995

In [28]:
## モデルBからhistoryのパラメータを抜き出す
history_coef <- df_results %>%
  filter(model_index == "reg_B",
         term == "history") %>%
  pull(estimate)
history_coef

[1] 0.001029897

In [29]:
## OVBの確認
OVB <- history_coef*treatment_coef[3]
coef_gap <- treatment_coef[1] - treatment_coef[2]
OVB # beta_2*gamma_1
coef_gap # alpha_1 - beta_1

[1] 0.02805398

[1] 0.02805398

以上、実装の理解のほうが難しいR独特の書き方編でした。

最後に入れてはいけない変数を入れてみる(Post treatment bias)

In [30]:
# (10) 入れてはいけない変数を入れてみる
#visitとtreatmentとの相関
cor_visit_treatment <- lm(data = biased_data,
                          formula = treatment ~ visit + channel + recency + history) %>%
  tidy()

# visitを入れた回帰分析を実行
bad_control_reg <- lm(data = biased_data,
                      formula = spend ~ treatment + channel + recency + history + visit) %>%
  tidy()

cor_visit_treatment %>% 
    filter(term=="visit") %>%
    select(term,estimate) #visitの係数 treatmentに相関が強いから分析に入れたくなるが...?

bad_control_reg %>%
    filter(term %in% c('treatment', 'visit')) %>%
    select(term,estimate) #treatmentの効果料が大幅に少なく見積もられる結果に！

term,estimate
<chr>,<dbl>
visit,0.1439984


term,estimate
<chr>,<dbl>
treatment,0.2939911
visit,7.1640215
